<a href="https://colab.research.google.com/github/camillan/llm-learning/blob/main/summarization_of_microplastics_articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
pip install transformers beautifulsoup4 requests biopython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.9 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup

def get_article_text(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, "html.parser")
        # Combine all paragraph tags into one string
        paragraphs = [p.get_text() for p in soup.find_all("p")]
        return " ".join(paragraphs)
    except Exception as e:
        print(f"❌ Error fetching {url}: {e}")
        return ""


In [17]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from Bio import Entrez

# ===== SETUP =====
Entrez.email = "cjn250@gmail.com"  # Replace with your actual email (required by NCBI)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# ===== FUNCTIONS =====

def extract_article_text(url):
    """Scrape visible text from a news or research article page."""
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = [p.get_text() for p in soup.find_all('p')]
        return " ".join(paragraphs)
    except Exception as e:
        print(f"❌ Error fetching {url}: {e}")
        return ""

def fetch_pubmed_abstract(pmid):
    """Fetch abstract text from PubMed using Entrez API."""
    try:
        handle = Entrez.efetch(db="pubmed", id=pmid, rettype="abstract", retmode="text")
        return handle.read()
    except Exception as e:
        print(f"❌ Error fetching PubMed {pmid}: {e}")
        return ""

def summarize_text(text, max_len=100, min_len=30):
    """Summarize a given block of text using a pre-trained model."""
    if not text.strip():
        return ""
    text = text[:4000]  # Truncate for model input size
    try:
        summary = summarizer(text, max_length=max_len, min_length=min_len, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        print(f"❌ Error summarizing: {e}")
        return ""

def summarize_all(web_urls, pubmed_pmids):
    """Process all articles and return individual and combined summaries."""
    all_summaries = []

    # Process web articles
    for url in web_urls:
        print(f"🌐 Scraping and summarizing: {url}")
        article_text = extract_article_text(url)
        summary = summarize_text(article_text)
        if summary:
            all_summaries.append(f"From {url}:\n{summary}\n")

    # Process PubMed abstracts
    for pmid in pubmed_pmids:
        print(f"🧬 Fetching and summarizing PubMed ID {pmid}")
        abstract = fetch_pubmed_abstract(pmid)
        summary = summarize_text(abstract)
        if summary:
            all_summaries.append(f"From PubMed ID {pmid}:\n{summary}\n")

    # Combine all summaries into one mega-summary
    combined_text = " ".join(all_summaries)
    print("\n🧠 Generating meta-summary of all sources...")
    final_summary = summarize_text(combined_text, max_len=250, min_len=80)

    return final_summary, all_summaries

# ===== RUN THIS PART =====

# Example list of sources
web_urls = [
    "https://www.eatingwell.com/how-to-limit-microplastics-in-your-food-11713723",
    "https://www.weforum.org/stories/2025/04/impact-microplastics-environment-health/",
    "https://marinedebris.noaa.gov/what-marine-debris/microplastics",
    "https://en.wikipedia.org/wiki/Microplastics",
    "https://www.ucsf.edu/news/2024/02/427161/how-to-limit-microplastics-dangers"
]

pubmed_urls = [
    "PMC9920460",
    "PMC10151227",
    "PMC6132564"
]

# Generate summaries
meta_summary, summaries = summarize_all(web_urls, pubmed_pmids)

# Print results
print("\n📄 INDIVIDUAL SUMMARIES:")
for s in summaries:
    print(s)

print("\n🧾 FINAL META-SUMMARY:")
print(meta_summary)


Device set to use cuda:0


🌐 Scraping and summarizing: https://www.eatingwell.com/how-to-limit-microplastics-in-your-food-11713723
🌐 Scraping and summarizing: https://www.weforum.org/stories/2025/04/impact-microplastics-environment-health/


Your max_length is set to 100, but your input_length is only 84. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)


🌐 Scraping and summarizing: https://marinedebris.noaa.gov/what-marine-debris/microplastics
🌐 Scraping and summarizing: https://en.wikipedia.org/wiki/Microplastics
🌐 Scraping and summarizing: https://www.ucsf.edu/news/2024/02/427161/how-to-limit-microplastics-dangers
🧬 Fetching and summarizing PubMed ID 37675473
🧬 Fetching and summarizing PubMed ID 37038415

🧠 Generating meta-summary of all sources...

📄 INDIVIDUAL SUMMARIES:
From https://www.eatingwell.com/how-to-limit-microplastics-in-your-food-11713723:
Microplastics are tiny plastic particles less than 5 millimeters in diameter. The majority of plastic waste is left to accumulate in landfills and the environment. There’s not enough evidence to directly link microplastics to a disease. Follow these steps to reduce your exposure.

From https://www.weforum.org/stories/2025/04/impact-microplastics-environment-health/:
Reference #18.984ddb17.1744503444.c448a619.https://errors.edgesuite.net/18. 984dd b17. 1744 503444. c448a 619.

From htt